In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/marykate/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import re

In [6]:
input_file = "angst-data/input2.txt"
file = open(input_file, "r") 
text = file.read() 
print('corpus length:', len(text))

corpus length: 494392


In [7]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
chars

total chars: 82


['\n',
 ' ',
 '!',
 '"',
 '$',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [8]:
## Chopping the text up into strings of 40 characters to train on
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 164784


In [9]:
## Vectorizing our sequences
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [10]:
## We're designing our model here- choosing the number of layers and such
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Showing the poetry quality at the end of each epoch
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        ## What we're doing is giving the bot 40 characters and asking
        ## it to predict the 41st character, over and over
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Here's where we train the model on our data
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
164736/164784 [============================>.] - ETA: 0s - loss: 2.2514
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " rise up from their old ailments,
their "
 rise up from their old ailments,
their have the was the will the shades and reach the she
where the would the would the reaked the will the were the will as the reeped

She seeples the shave the weat the shades
and the would the see the weet the weep the read

And the stare the reated the will the reated

I was the sun the sard as the start the wards
and the weat the weat the was a pilling the was the stare.

The soully the ward the we
----- diversity: 0.5
----- Generating with seed: " rise up from their old ailments,
their "
 rise up from their old ailments,
their she are the hat in the pain. 
The are the were beat the pace be age
cater the well the reppetused the bight the reeped serced of a procould the have all the bodest and bright the breaked in what the suself as my all

Admit it: elline. 
Greepering silver hiving of memory dows fear cullmby
like ever retrow a dreamed of or silver dglemperence weas?

We urus or lioving ant another.

lands and Soun, lefon zod
w Eatly way. The greed, rescar pain
like pinco in flumily culm
to a life . & dogs their beating his es up our sligt.
Spelar, isnored so oge of viem. 
The morimemb unterpalt of and blurder your roins arnert
We sumpay in 
----- diversity: 1.2
----- Generating with seed: "d almost instantly away.
VII.
Admit it: "
d almost instantly away.
VII.
Admit it: larred is liss of forescone
across head, you hand word by itlusely hafter-cold.  cierr,
you Imara, , fiintionan to its eliche

Could moment to listaning more better,
nother-in dus   incher, diciiftlar
waye, eyNumoticates thinger doled,
ever nrupart
 knome,  Ord, war loft, vomys. Sudelal imusal ry,
Who lots authond sage, all their Avasbad ffolds, She love to do ng decky.
my children when  Habries.

164784/164784 [==============================] - 248s 2m

Oh! silent the horror, and fierce the show and strangers of the strangers of the street
in the seas the should be a ture. 

The part of the should have the sea should as the sound,
the show are the show and was the sea, 
The secked in the sun speak and some the should see the sound,
the street the show and sometimes and the shall 

The show with the should have the back in the sunset and so wands and fall of the street
the sunse
----- diversity: 0.5
----- Generating with seed: "s done;
Oh! silent the horror, and fierc"
s done;
Oh! silent the horror, and fierce of the can small pain
and the should grown and the grast and the grass 
The sease of the grain that happy in one sears
with the same as with the world so wonder that hat inside grass.

And the stopped to a hands the show and its turned make. 

The set sweet light and intensiarrion
as I should all the should be are rest to be truttered agones,
but the can halls a things, which shall of one dream 
----- diversity: 1.0
----- Generat

it all the scraze, and trees,
    And manil those bratureput for me.
The mother grief I retal, on, the getting sunre wared it heart borrowscar, erot?
on his hand on the shotter louds mains are fegeered

I stere unget repeurs, which to like or wring
    in-icticle and the boust
I live nout like full gathow one stands

I fell
----- diversity: 1.2
----- Generating with seed: "to the burned arcade
 where you said I s"
to the burned arcade
 where you said I steery planny herilove, scicy.

thank have here, he would narludgers togsone
Lights, are you,once. And sealing octurn 

livilard towched word,
In paid puproated stage, alperpullryGy flosely,
   Srillr manIwfmard.
Sianoc eckmove,
municle of the eclous your wingpsed in the stainthy.

Fat, wothin ussic phimorors
mare clelly, with.
And mashlows concenciouts of allny up the treat
opplade
lamphdilty on t
164784/164784 [==============================] - 247s 1ms/step - loss: 1.6194
Epoch 12/60
164736/164784 [============================>.] - ET

emerging as the street is a set the street 
The sun who knew the street is the street 

how the streed is the soul me in the shaded to the street 

to the streed is a soul and the street in the sun, 

the stream the street in the south to see the street 
The sky and when the sun and with the street 

to the street is father of the seat and with the street 
And the streets and when the shaded to the streets and th
----- diversity: 0.5
----- Generating with seed: "ng into gasoline fires,
emerging as the "
ng into gasoline fires,
emerging as the shade the silenge to trandic.
Some to the shaded the world At of growing strain sost
it stands the proscers 
that was to real of the whose is to be light,
or his drives even the same who loved to seemed the Hears,

to what the light the skins and bead of the was to the death of the my the stream one perception
I was stood that is a sould of someone to predished to we this troused
To be summed it o
----- diversity: 1.0
----- Generating with seed: "

...

Sunred us and move alien 
in a studders the night,  I cannot been.

I said me wait higher Alls in of,
There prushing, day, see
----- diversity: 1.2
----- Generating with seed: " blown upon
the unforgiving surface,
but"
 blown upon
the unforgiving surface,
but aunrides, air. I vacing mortating needereseses,
    And us
for his people me,
whather conspite nonkle

by actericated mavuleth belly unolling
of to, and powely Idens. studumeag
Strothers. the haisg futudious naver hath. This your night svaint.


Id expents unpantibles high strong the others ow crowlds eater's shart, frack 
Maor fire speakles netith, some away hong boy
 we guean me too. Cara,
As t
164784/164784 [==============================] - 245s 1ms/step - loss: 1.5490
Epoch 28/60
164736/164784 [============================>.] - ETA: 0s - loss: 1.5455
----- Generating text after Epoch: 27
----- diversity: 0.2
----- Generating with seed: "pe 
If honour find him in the wintry bla"
pe 
If honour find him in the wintry bland of th

They was the strange the streated the streething the bones of the street
of the black she was and be a strange fire.

And the broke an anterning of the world.
The black in the strange the streets of the story be the breathed
the morning the st
----- diversity: 0.5
----- Generating with seed: "ll, 
Up goes her hand, and off she slips"
ll, 
Up goes her hand, and off she slips them 
He was still their can of the grass
And the heaven the finger black of the counter.

And the strange to light to chance that has think men to stays
To the wreee the way the heart of the finger


And in the dead and where for the world,
and what he feel the chines, the larking house and and great seen of loving bears 
So in the could have the tarry of bed the see

they say the coust where th
----- diversity: 1.0
----- Generating with seed: "ll, 
Up goes her hand, and off she slips"
ll, 
Up goes her hand, and off she slips as . . . stall 
Sifggaftiss from painning as man was 
And all there that tare, they ins. 


My chold upon that factive floxam turned inty gaudient she is a mingerned novelticalctirics,

That sound, that was grand, he knew to see tow.
say the upsting of minochway it was
----- diversity: 1.2
----- Generating with seed: "roved as vain  
Till, spurred anew by fe"
roved as vain  
Till, spurred anew by feir
She strangers of young at the urn.
year that werefnels you bottle own nothing.
Brown a shaughts, intersolss toved the drugg
entures-side like a rilly.
he recark which wish
obbyque but centers
    isly, a thickled  us for alsone
wretching to cuffusling
the sktweep.

And he to the baugens's phintal, which troud, even 

Afternal and drops. Har ago.l the another swolles
She say rest 

white-the gla
164784/164784 [==============================] - 253s 2ms/step - loss: 1.5295
Epoch 35/60
164736/164784 [============================>.] - ETA: 0s - loss: 1.5296
----- Generating text after Epoch: 34
----- diversity: 0.2
----- Generating with seed: "anhungered of life as a tiger from toil

the stranger of the story of the world stands of the story the story to the story to the story to the story that was and the story to the story the stranger in the story that of the story the story to see
the shadow and water and the strangers of the story to the story to the story to see
the stranger and stars the
----- diversity: 0.5
----- Generating with seed: "past, lots of news items
about what was "
past, lots of news items
about what was think in the and stay, 
And I taul for the stars and between the counter of the branches

where the shrettering the pain, if we could so and the door because she water of the story the pain.

I was to feel her mouth
of the whisper of the mouth and the moon and still of his life,

The first who have a worm of my branged the skin

And while the sun, the plate on the sandy state,
I walked to the coff
----- diversity: 1.0
----- Generating with seed: "past, lots of news items
about what was "
past, lots of news items
about what was live, some bound a

Sucruclity with pider rambet-knon, the oed to dream, or should where we do the spiper.

Her brate of floor of a past-leafe that relage.
And so ining or fire

In sun mixeency ctaring in jech diosepswith. 
Some doely lung a
----- diversity: 1.2
----- Generating with seed: "gs have been done to us that can never b"
gs have been done to us that can never beepond.
Bitture kink fear, the where spiding, pecking bluey
In the doom came for
not heavy just us!
When everyor you-bualcehold. 
I hard, at aze become
I tell? Old killscomators tlen,
Flugt in. 

Chilly like a tweny's an some cut. facts up the kendealb
To your aimarg-triring
without wintew-shaked his bed, left
lost procking of if twent; sensced the stone: 
Aldiation in hold Ianqulachallony Wanden 
164784/164784 [==============================] - 246s 1ms/step - loss: 1.5199
Epoch 42/60
164736/164784 [============================>.] - ETA: 0s - loss: 1.5161
----- Generating text after Epoch: 41
----- diversity: 0.2
----- Generating with se

In [ ]:
model.save('angst2.h5')

In [25]:
## Here is where you can actually use the model on a given piece of text!

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = 'There once was a cool guy named Cam.....'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(800):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- diversity: 0.2
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam......
 the bestorment the stream and the street 
The warm her back and the common of the distant 
The we've we have seen the sun was a strong, 
And when the country of the shadows and the common of the wood.
The warm the shakened of the seasong and the sea
and brother was the same who come, 
And the said west the sun was the stream and the finged the sea, 
And the sun was a guess and the cornial streams and the sun for the stare 
The bed the sun was the morning and the sun is not my spitted and the common of the distant 
The field the mountains of the colored by the work,
and the sun will to the sun was a stand 
The said, and the water of the sea, 
And born the sun was the sun is not flowers of the morning of the work,
the seating the room of the stone of the works
and the warmied and the way 
----- diversity: 0.5
----- Generating with seed: "There once was a 

/home/Randus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


Beauth like the small greet, 
The resount of the shadows the conkering to my many old trees,
the bold had the mountains of every pain 
Who was the heart he says, and the rain and brown 
The pease and the sun the way the coon light of the boat.
And for the bones of bedian from a shakes the sea, 
See a callar lived it see the wain down to the wells of it,
angual of the columf in the work,
an of the flack on the flowers of the woods
The sun they said we should be and is well worsed or the pain and his rise and soul. 
And flass, or by the turfees of day of the sea
----- diversity: 1.0
----- Generating with seed: "There once was a cool guy named Cam....."
There once was a cool guy named Cam.....my pourds though to charded,
manianing we, greit is theory.
Soldy a towning, whan immortain neat-angine flower
wabun young mortale at onatio tongue use again mudes years, hie drawnce
There, in this lights that since I waking friend . .
For let many lockt trees they closer mort greetic blue, 
With str